# Создание глубокой нейронной сети шаг за шагом

Добро пожаловать! Ранее вы обучили двухслойную нейронную сеть (с одним скрытым слоем). На этой неделе вы построите глубокую нейронную сеть с любым количеством слоев!

- В этом блокноте вы реализуете все функции, необходимые для построения глубокой нейронной сети.
- В следующем задании вы будете использовать эти функции для построения глубокой нейронной сети для классификации изображений.

**После этого задания вы сможете:**  
- Использовать нелинейные функции, такие как ReLU, чтобы улучшить свою модель.
- Построить более глубокую нейронную сеть (с более чем одним скрытым слоем)
- Реализуете простой в использовании класс нейронной сети.

**Обозначение**:  
- Верхний индекс $[l]$ обозначает величину, связанную со слоем $l$.
     - Пример: $a^{[L]}$ — это активация слоя $L$. $W^{[L]}$ и $b^{[L]}$ — параметры слоя $L$.
- Надстрочный индекс $(i)$ обозначает величину, связанную с примером $i$.
     - Пример: $x^{(i)}$ — это обучающий пример $i$.
- Нижний индекс $i$ обозначает $i$-й элемент вектора.
     - Пример: $a^{[l]}_i$ обозначает $i$-й запись активаций слоя $l$).

Давайте начнем!

## 1 - Пакеты

Давайте сначала импортируем все пакеты, которые вам понадобятся во время этого задания.
- [numpy](www.numpy.org) — основной пакет для научных вычислений на Python.
- [matplotlib](http://matplotlib.org) — библиотека для построения графиков на Python.
- dnn_utils предоставляет некоторые необходимые функции для этого блокнота.
- testCases предоставляет несколько тестовых примеров для оценки правильности ваших функций.
- np.random.seed(1) используется для обеспечения согласованности всех случайных вызовов функций. Это поможет нам оценить вашу работу. Пожалуйста, не меняйте seed.

In [8]:
import numpy as np
import h5py
import matplotlib.pyplot as plt
from testCases_v4 import *
from dnn_utils_v2 import sigmoid, sigmoid_backward, relu, relu_backward

%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2

np.random.seed(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 2 – Краткое описание задания

Чтобы построить свою нейронную сеть, вы будете реализовывать несколько «вспомогательных функций». Эти вспомогательные функции будут использоваться в следующем задании для построения двухслойной нейронной сети и нейронной сети из L слоев. Каждая небольшая вспомогательная функция, которую вы реализуете, будет иметь подробные инструкции, которые проведут вас через необходимые шаги. Вот план этого задания, вы будете:

- Инициализируете параметры для двухслойной сети и $L$-слойной нейронной сети.
- Реализуете модуль прямого распространения (показан фиолетовым цветом на рисунке ниже).
      - Завершите ЛИНЕЙНУЮ часть шага прямого распространения слоя (в результате получим $Z^{[l]}$).
      - мы предоставляем Вам функцию АКТИВАЦИИ (relu/sigmoid).
      - Объедините два предыдущих шага в новую прямую функцию [LINEAR->ACTIVATION].
      - Соедините функцию прямого распространения [LINEAR->RELU] L-1 раз (для слоев с 1 по L-1) и добавите [LINEAR->SIGMOID] в конце (для последнего слоя $L$). Это дает вам новую функцию L_model_forward.
- Подсчитаете потери.
- Реализуете модуль обратного распространения (обозначен красным на рисунке ниже).
     - Завершите ЛИНЕЙНУЮ часть этапа обратного распространения слоя.
     - Мы даем вам градиент функции АКТИВАЦИИ (relu_backward/sigmoid_backward)
     - Объедините два предыдущих шага в новую обратную функцию [LINEAR->ACTIVATION].
     - Соедините [LINEAR->RELU] в обратном проходе L-1 раз и добавите [LINEAR->SIGMOID] в новой функции L_model_backward.
- Наконец сделаете один шаг обучения - обновите параметры.

<img src="images/final Outline.png" style="width:800px;height:500px;">
<caption><center><bold> Рис. 1 </bold></center></caption><br>


**Обратите внимание**, что каждой прямой функции соответствует обратная функция. Вот почему на каждом этапе вашего модуля пересылки вы будете сохранять некоторые значения в кеше. Кэшированные значения полезны для вычисления градиентов. Затем в модуле обратного распространения ошибки вы будете использовать кеш для расчета градиентов. Это задание покажет вам, как именно выполнить каждый из этих шагов.

## 3 - Инициализация

Вы напишете две вспомогательные функции, которые будут инициализировать параметры вашей модели. Первая функция будет использоваться для инициализации параметров двухслойной модели. Вторая обобщит  процесс инициализации на $L$-слоёв .

### 3.1 - 2-слойная нейронная сеть

**Упражнение**. Создайте и инициализируйте параметры двухслойной нейронной сети.

**Инструкции**:  
- Структура модели: *LINEAR -> RELU -> LINEAR -> SIGMOID*.
- Используйте случайную инициализацию весовых матриц. Используйте `np.random.randn(shape)*0.01` с правильными размерами.
- Используйте нулевую инициализацию для смещений. Используйте `np.zeros(shape)`.

In [9]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    parameters -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(1)
    
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = np.random.randn(n_h,n_x)*0.01
    b1 = np.random.randn(n_h,1)*0.01
    W2 = np.random.randn(n_y,n_h)*0.01
    b2 = np.random.randn(n_y,1)*0.01
    ### END CODE HERE ###
    
    assert(W1.shape == (n_h, n_x))
    assert(b1.shape == (n_h, 1))
    assert(W2.shape == (n_y, n_h))
    assert(b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters    

In [10]:
parameters = initialize_parameters(3,2,1)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]]
b1 = [[ 0.01744812]
 [-0.00761207]]
W2 = [[ 0.00319039 -0.0024937 ]]
b2 = [[0.01462108]]


**Ожидаемый вывод**:
       
<table style="width:80%">
  <tr>
    <td> **W1** </td>
    <td> [[ 0.01624345 -0.00611756 -0.00528172]
 [-0.01072969  0.00865408 -0.02301539]] </td> 
  </tr>

  <tr>
    <td> **b1**</td>
    <td>[[ 0.]
 [ 0.]]</td> 
  </tr>
  
  <tr>
    <td>**W2**</td>
    <td> [[ 0.01744812 -0.00761207]]</td>
  </tr>
  
  <tr>
    <td> **b2** </td>
    <td> [[ 0.]] </td> 
  </tr>
  
</table>

### 3.2 - L-слойная нейронная сеть

Инициализация более глубокой L-слойной нейронной сети сложнее, поскольку здесь гораздо больше весовых матриц и векторов смещения. При заполнении «initialize_parameters_deep» вы должны убедиться, что ваши размеры совпадают на всех слоях. Напомним, что $n^{[l]}$ — это количество единиц в слое $l$. Например, если размер входных данных $X$ равен $(12288, 209)$ (с $m=209$ примерами), то:

<table style="width:100%">
    <tr>
        <td>  </td> 
        <td> **Shape of W** </td> 
        <td> **Shape of b**  </td> 
        <td> **Activation** </td>
        <td> **Shape of Activation** </td> 
    <tr>
    <tr>
        <td> **Layer 1** </td> 
        <td> $(n^{[1]},12288)$ </td> 
        <td> $(n^{[1]},1)$ </td> 
        <td> $Z^{[1]} = W^{[1]}  X + b^{[1]} $ </td> 
        <td> $(n^{[1]},209)$ </td> 
    <tr>
    <tr>
        <td> **Layer 2** </td> 
        <td> $(n^{[2]}, n^{[1]})$  </td> 
        <td> $(n^{[2]},1)$ </td> 
        <td>$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$ </td> 
        <td> $(n^{[2]}, 209)$ </td> 
    <tr>
       <tr>
        <td> $\vdots$ </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$  </td> 
        <td> $\vdots$</td> 
        <td> $\vdots$  </td> 
    <tr>
    <tr>
        <td> **Layer L-1** </td> 
        <td> $(n^{[L-1]}, n^{[L-2]})$ </td> 
        <td> $(n^{[L-1]}, 1)$  </td> 
        <td>$Z^{[L-1]} =  W^{[L-1]} A^{[L-2]} + b^{[L-1]}$ </td> 
        <td> $(n^{[L-1]}, 209)$ </td> 
    <tr>
    <tr>
        <td> **Layer L** </td> 
        <td> $(n^{[L]}, n^{[L-1]})$ </td> 
        <td> $(n^{[L]}, 1)$ </td>
        <td> $Z^{[L]} =  W^{[L]} A^{[L-1]} + b^{[L]}$</td>
        <td> $(n^{[L]}, 209)$  </td> 
    <tr>
</table>

Помните, что когда мы вычисляем $W X + b$ в Python, осуществляется broadcasting. Например, если:

$$ W = \begin{bmatrix}
    j  & k  & l\\
    m  & n & o \\
    p  & q & r 
\end{bmatrix}\;\;\; X = \begin{bmatrix}
    a  & b  & c\\
    d  & e & f \\
    g  & h & i 
\end{bmatrix} \;\;\; b =\begin{bmatrix}
    s  \\
    t  \\
    u
\end{bmatrix}\tag{2}$$

Тогда $WX + b$ будет:

$$ WX + b = \begin{bmatrix}
    (ja + kd + lg) + s  & (jb + ke + lh) + s  & (jc + kf + li)+ s\\
    (ma + nd + og) + t & (mb + ne + oh) + t & (mc + nf + oi) + t\\
    (pa + qd + rg) + u & (pb + qe + rh) + u & (pc + qf + ri)+ u
\end{bmatrix}\tag{3}  $$

**Упражнение**. Реализуйте инициализацию L-слойной нейронной сети.

**Инструкции**:  
- Структура модели: *[LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID*. То есть она имеет $L-1$ слоев, использующих функцию активации ReLU, за которыми следует выходной слой с функцией активации сигмоидой.
- Используйте случайную инициализацию весовых матриц. Используйте `np.random.randn(shape) * 0.01`.
- Для смещений используйте инициализацию нулями. Используйте `np.zeros(shape)`.
- Мы будем хранить $n^{[l]}$, количество нейронов в разных слоях, в переменной `layer_dims`. Например, `layer_dims` для «Модели классификации плоских данных» с прошлой недели был бы [2,4,1]: было два входа, один скрытый слой с 4 скрытыми единицами и выходной слой с 1 выходной единицей. Таким образом, форма `W1` была (4,2), `b1` была (4,1), `W2` была (1,4) и `b2` была (1,1). Теперь вы обобщите это на слои $L$!
- Вот реализация для $L=1$ (однослойная нейронная сеть). Это должно вдохновить вас на реализацию общего случая (нейронная сеть L-слоя).
```python
    if L == 1:
        parameters["W" + str(L)] = np.random.randn(layer_dims[1], layer_dims[0]) * 0.01
        parameters["b" + str(L)] = np.zeros((layer_dims[1], 1))
```

In [23]:
# GRADED FUNCTION: initialize_parameters_deep

def initialize_parameters_deep(layer_dims):
    """
    Arguments:
    layer_dims -- python array (list) containing the dimensions of each layer in our network
    
    Returns:
    parameters -- python dictionary containing your parameters "W1", "b1", ..., "WL", "bL":
                    Wl -- weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl -- bias vector of shape (layer_dims[l], 1)
    """
    
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        ### START CODE HERE ### (≈ 2 lines of code)
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        ### END CODE HERE ###
        
        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

        
    return parameters

In [24]:
parameters = initialize_parameters_deep([5,4,3])
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

W1 = [[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]
b1 = [[0.]
 [0.]
 [0.]
 [0.]]
W2 = [[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]
b2 = [[0.]
 [0.]
 [0.]]


**Ожидаемый выход**:
       
<table style="width:80%">
  <tr>
    <td> **W1** </td>
    <td>[[ 0.01788628  0.0043651   0.00096497 -0.01863493 -0.00277388]
 [-0.00354759 -0.00082741 -0.00627001 -0.00043818 -0.00477218]
 [-0.01313865  0.00884622  0.00881318  0.01709573  0.00050034]
 [-0.00404677 -0.0054536  -0.01546477  0.00982367 -0.01101068]]</td> 
  </tr>
  
  <tr>
    <td>**b1** </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
  
  <tr>
    <td>**W2** </td>
    <td>[[-0.01185047 -0.0020565   0.01486148  0.00236716]
 [-0.01023785 -0.00712993  0.00625245 -0.00160513]
 [-0.00768836 -0.00230031  0.00745056  0.01976111]]</td> 
  </tr>
  
  <tr>
    <td>**b2** </td>
    <td>[[ 0.]
 [ 0.]
 [ 0.]]</td> 
  </tr>
  
</table>

## 4 - Модуль прямого распространения (forward propagation)

### 4.1 - Линейный forward
Теперь, когда вы инициализировали свои параметры, вы займетесь модулем прямого распространения (forward propagation). Вы начнете с реализации некоторых основных функций, которые вы будете использовать позже при реализации модели. Выполните три функции в следующем порядке:

- LINEAR
- LINEAR -> ACTIVATION, где ACTIVATION будет либо ReLU, либо Sigmoid.
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID (вся модель)

Линейный модуль прямого распространения (векторизованный по всем примерам) вычисляет следующие уравнения:

$$Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}\tag{4}$$

где $A^{[0]} = X$. 

**Упражнение**. Постройте линейную часть прямого распространения сигнала.

**Напоминание**:  
Математическое представление этого модуля: $Z^{[l]} = W^{[l]}A^{[l-1]} +b^{[l]}$. Вам также может пригодиться `np.dot()`. Если ваши размеры не совпадают, может помочь вывод на печать `W.shape`.

In [13]:
# GRADED FUNCTION: linear_forward

def linear_forward(A, W, b):
    """
    Implement the linear part of a layer's forward propagation.

    Arguments:
    A -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)

    Returns:
    Z -- the input of the activation function, also called pre-activation parameter 
    cache -- a python dictionary containing "A", "W" and "b" ; stored for computing the backward pass efficiently
    """
    
    ### START CODE HERE ### (≈ 1 line of code)
    Z = np.dot(W,A)+b
    ### END CODE HERE ###
    
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [14]:
A, W, b = linear_forward_test_case()

Z, linear_cache = linear_forward(A, W, b)
print("Z = " + str(Z))

Z = [[ 3.26295337 -1.23429987]]


**Ожидаемый вывод**:

<table style="width:35%">
  
  <tr>
    <td> **Z** </td>
    <td> [[ 3.26295337 -1.23429987]] </td> 
  </tr>
  
</table>

### 4.2 - Linear-Activation Forward

В этом блокноте вы будете использовать две функции активации:

- **Sigmoid**: $\sigma(Z) = \sigma(W A + b) = \frac{1}{ 1 + e^{-(W A + b)}}$. Мы предоставили вам функцию «Sigmoid». Эта функция возвращает **два** элемента: значение активации «a» и «кэш», содержащий «Z» (это то, что мы передадим в соответствующую обратную функцию). Чтобы использовать его, вы можете просто вызвать:
``` python
A, activation_cache = sigmoid(Z)
```

- **ReLU**: Математическая формула ReLu: $A = RELU(Z) = max(0, Z)$. Мы предоставили вам функцию relu. Эта функция возвращает **два** элемента: значение активации «A» и «кэш», содержащий «Z» (это то, что мы передадим в соответствующую обратную функцию). Чтобы использовать его, вы можете просто вызвать:
``` python
A, activation_cache = relu(Z)
```

Для большего удобства вы сгруппируете две функции (Линейная и Активация) в одну функцию (ЛИНЕЙНАЯ->АКТИВАЦИЯ). Т.е. вы реализуете функцию, которая выполняет ЛИНЕЙНЫЙ шаг, за которым следует шаг АКТИВАЦИЯ.

**Упражнение**: реализуйте метод прямого распространения *LINEAR->ACTIVATION*. Математическое соотношение: $A^{[l]} = g(Z^{[l]}) = g(W^{[l]}A^{[l-1]} +b^{[l]} )$, где активация "g" может быть sigmoid() или relu(). Используйте `linear_forward()` и правильную функцию активации.

In [19]:
# GRADED FUNCTION: linear_activation_forward

def linear_activation_forward(A_prev, W, b, activation):
    """
    Implement the forward propagation for the LINEAR->ACTIVATION layer

    Arguments:
    A_prev -- activations from previous layer (or input data): (size of previous layer, number of examples)
    W -- weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b -- bias vector, numpy array of shape (size of the current layer, 1)
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:
    A -- the output of the activation function, also called the post-activation value 
    cache -- a python dictionary containing "linear_cache" and "activation_cache";
             stored for computing the backward pass efficiently
    """
    
    if activation == "sigmoid":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = sigmoid(Z)
        ### END CODE HERE ###
    
    elif activation == "relu":
        # Inputs: "A_prev, W, b". Outputs: "A, activation_cache".
        ### START CODE HERE ### (≈ 2 lines of code)
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z)
        ### END CODE HERE ###
    
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

In [20]:
A_prev, W, b = linear_activation_forward_test_case()

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "sigmoid")
print("With sigmoid: A = " + str(A))

A, linear_activation_cache = linear_activation_forward(A_prev, W, b, activation = "relu")
print("With ReLU: A = " + str(A))

With sigmoid: A = [[0.96890023 0.11013289]]
With ReLU: A = [[3.43896131 0.        ]]


**Ожидаемый вывод**:
       
<table style="width:35%">
  <tr>
    <td> **With sigmoid: A ** </td>
    <td > [[ 0.96890023  0.11013289]]</td> 
  </tr>
  <tr>
    <td> **With ReLU: A ** </td>
    <td > [[ 3.43896131  0.        ]]</td> 
  </tr>
</table>


**Замечание**: В глубоком обучении вычисление "[LINEAR->ACTIVATION]" обычно считается одним слоем нейронной сети, а не двумя отдельными слоями.

### L-слойная модель

Для еще большего удобства при реализации нейронной сети с $L$-слоями вам понадобится функция, которая повторяет предыдущую (`linear_activation_forward` с RELU) $L-1$ раз, а затем одну `linear_activation_forward` с SIGMOID.

<img src="images/model_architecture_kiank.png" style="width:600px;height:300px;">
<caption><center><bold>Рисунок 2</bold>: *[LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID* model</center></caption><br>

**Упражнение**. Реализуйте функцию прямого распространения описанной выше модели.

**Инструкция**: В приведенном ниже коде переменная `AL` будет обозначать $A^{[L]} = \sigma(Z^{[L]}) = \sigma(W^{[L]} A ^{[L-1]} + b^{[L]})$. (Иногда это еще называют $\hat{Y}$.)

**Советы**:
- Используйте функции, которые вы написали ранее
- Используйте цикл for для репликации [LINEAR->RELU] (L-1) раз.
- Не забывайте следить за кешами в списке «кеш». Чтобы добавить новое значение `c` в `list`, вы можете использовать `list.append(c)`.

In [31]:
# GRADED FUNCTION: L_model_forward

def L_model_forward(X, parameters):
    """
    Implement forward propagation for the [LINEAR->RELU]*(L-1)->LINEAR->SIGMOID computation
    
    Arguments:
    X -- data, numpy array of shape (input size, number of examples)
    parameters -- output of initialize_parameters_deep()
    
    Returns:
    AL -- last post-activation value
    caches -- list of caches containing:
                every cache of linear_activation_forward() (there are L-1 of them, indexed from 0 to L-1)
    """

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    # Implement [LINEAR -> RELU]*(L-1). Add "cache" to the "caches" list.
    for l in range(1, L):
        A_prev = A 
        ### START CODE HERE ### (≈ 2 lines of code)
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], 'relu')
        caches.append(cache)
        ### END CODE HERE ###
    
    # Implement LINEAR -> SIGMOID. Add "cache" to the "caches" list.
    ### START CODE HERE ### (≈ 2 lines of code)
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)], parameters['b' + str(L)], 'sigmoid')
    caches.append(cache)
    ### END CODE HERE ###
    
    assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [32]:
X, parameters = L_model_forward_test_case_2hidden()
AL, caches = L_model_forward(X, parameters)
print("AL = " + str(AL))
print("Length of caches list = " + str(len(caches)))

AL = [[0.03921668 0.70498921 0.19734387 0.04728177]]
Length of caches list = 3


<table style="width:50%">
  <tr>
    <td> **AL** </td>
    <td > [[ 0.03921668  0.70498921  0.19734387  0.04728177]]</td> 
  </tr>
  <tr>
    <td> **Length of caches list ** </td>
    <td > 3 </td> 
  </tr>
</table>

Большой! Теперь у вас есть полная функция прямого распространения, которая принимает входные данные X и выводит вектор-строку $A^{[L]}$, содержащий ваши прогнозы. Она также записывает все промежуточные значения в «кеши». Используя $A^{[L]}$, вы можете вычислить ошибку (лосс) своих прогнозов.

## 5 - Функция ошибок

Теперь вы реализуете прямое и обратное распространение. Вам необходимо вычислить функцию ошибки, чтобы проверить, действительно ли ваша модель обучается.

**Упражнение**. Вычислите лосс-функцию кросс-энтропии $J$ по следующей формуле:

$$-\frac{1}{m} \sum\limits_{i = 1}^{m} (y^{(i)}\log\left(a^{[L] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[L](i)}\right)) \tag{7}$$


In [33]:
# GRADED FUNCTION: compute_cost

def compute_cost(AL, Y):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    ### START CODE HERE ### (≈ 1 lines of code)
    cost = (-1/Y.shape[1])*np.sum(Y*np.log(AL)+(1-Y)*np.log(1-AL))
    ### END CODE HERE ###
    
    cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    assert(cost.shape == ())
    
    return cost

In [34]:
Y, AL = compute_cost_test_case()

print("cost = " + str(compute_cost(AL, Y)))

cost = 0.41493159961539694


**Ожидаемый выход**:

<table>
    <tr>
    <td>**cost** </td>
    <td> 0.41493159961539694</td> 
    </tr>
</table>

## 6 - Модуль обратного распространения (backward propagation)

Как и в случае с прямым распространением сигнала (forward propagation), вы реализуете вспомогательные функции для обратного распространения ошибки (backward propagation). Помните, что обратное распространение используется для расчета градиента функции потерь относительно параметров.

**Напоминание**:  
<img src="images/backprop_kiank.png" style="width:650px;height:250px;">
<caption><center><bold>Рис. 3</bold>: Прямое и обратное распространение для *LINEAR->RELU->LINEAR->SIGMOID* <br> *Фиолетовые блоки представляют прямое распространение, а красные блоки — обратное распространение.* </center></caption>

<!--
Для тех из вас, кто является экспертом в матанализе (вам это не обязательно), можно использовать цепное правило исчисления для получения производной функции потерь $\mathcal{L}$ по $z. ^{[1]}$ в двухслойной сети следующим образом:
$$\frac{d \mathcal{L}(a^{[2]},y)}{{dz^{[1]}}} = \frac{d\mathcal{L}(a^{[2]},y)}{{da^{[2]}}}\frac{{da^{[2]}}}{{dz^{[2]}}}\frac{{dz^{[2]}}}{{da^{[1]}}}\frac{{da^{[1]}}}{{dz^{[1]}}} \tag{8} $$

Чтобы вычислить градиент $dW^{[1]} = \frac{\partial L}{\partial W^{[1]}}$, вы используете предыдущее цепное правило и делаете $dW^{[1 ]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial W^{[1]}}$. Во время обратного распространения ошибки на каждом этапе вы умножаете текущий градиент на градиент, соответствующий конкретному слою, чтобы получить желаемый градиент.

Аналогично, чтобы вычислить градиент $db^{[1]} = \frac{\partial L}{\partial b^{[1]}}$, вы используете предыдущее цепное правило и делаете $db^{ [1]} = dz^{[1]} \times \frac{\partial z^{[1]} }{\partial b^{[1]}}$.

Вот почему мы говорим об **обратном распространении**.
!-->

Теперь, как и в случае с прямым распространением, построим обратное распространение в три этапа:
- LINEAR backward
- LINEAR -> ACTIVATION backward, где ACTIVATION вычисляет производную активации ReLU или SIGMOID.
- [LINEAR -> RELU] $\times$ (L-1) -> LINEAR -> SIGMOID backward (вся модель)

### 6.1 -  Linear backward

Для слоя $l$ линейная часть равна: $Z^{[l]} = W^{[l]} A^{[l-1]} + b^{[l]}$ (с последующей активацией).

Предположим, вы уже вычислили производную $dZ^{[l]} = \frac{\partial \mathcal{L} }{\partial Z^{[l]}}$. Вы хотите получить $(dW^{[l]}, db^{[l]} dA^{[l-1]})$.

<img src="images/linearback_kiank.png" style="width:250px;height:300px;">
<caption><center> **Figure 4** </center></caption>

Три выхода $(dW^{[l]}, db^{[l]}, dA^{[l]})$ вычисляются с использованием входных данных $dZ^{[l]}$. Вот формулы, которые вы можете использовать:
$$ dW^{[l]} = \frac{\partial \mathcal{L} }{\partial W^{[l]}} = \frac{1}{m} dZ^{[l]} A^{[l-1] T} \tag{8}$$
$$ db^{[l]} = \frac{\partial \mathcal{L} }{\partial b^{[l]}} = \frac{1}{m} \sum_{i = 1}^{m} dZ^{[l](i)}\tag{9}$$
$$ dA^{[l-1]} = \frac{\partial \mathcal{L} }{\partial A^{[l-1]}} = W^{[l] T} dZ^{[l]} \tag{10}$$


**Упражнение**. Используйте три приведенные выше формулы, чтобы реализовать linear_backward().

In [35]:
# GRADED FUNCTION: linear_backward

def linear_backward(dZ, cache):
    """
    Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    A_prev, W, b = cache
    m = A_prev.shape[1]

    ### START CODE HERE ### (≈ 3 lines of code)
    dW = 1 / m * (np.dot(dZ,A_prev.T))
    db = 1 / m * (np.sum(dZ,axis = 1,keepdims = True))
    dA_prev = np.dot(W.T,dZ)
    ### END CODE HERE ###
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [36]:
# Set up some test inputs
dZ, linear_cache = linear_backward_test_case()

dA_prev, dW, db = linear_backward(dZ, linear_cache)
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

dA_prev = [[ 0.51822968 -0.19517421]
 [-0.40506361  0.15255393]
 [ 2.37496825 -0.89445391]]
dW = [[-0.10076895  1.40685096  1.64992505]]
db = [[0.50629448]]


**Ожидаемый вывод**: 

<table style="width:90%">
  <tr>
    <td> **dA_prev** </td>
    <td > [[ 0.51822968 -0.19517421]
           [-0.40506361  0.15255393]
           [ 2.37496825 -0.89445391]] 
    </td> 
  </tr> 
  <tr>
    <td> **dW** </td>
    <td > [[-0.10076895  1.40685096  1.64992505]] </td> 
  </tr> 
  <tr>
    <td> **db** </td>
    <td> [[ 0.50629448]] </td> 
  </tr> 
</table>



### 6.2 - Linear-Activation backward

Далее вы создадите функцию, которая объединит две вспомогательные функции: **`linear_backward`** и обратный шаг для активации **`linear_activation_backward`**.

Чтобы помочь вам реализовать «linear_activation_backward», мы предоставили две обратные функции:
- **`sigmoid_backward`**: реализует обратное распространение для модуля SIGMOID. Вы можете вызвать его следующим образом:

``` питон
dZ = sigmoid_backward(dA, active_cache)
```

- **`relu_backward`**: реализует обратное распространение для модуля RELU. Вы можете вызвать его следующим образом:

``` питон
dZ = relu_backward(dA, active_cache)
```

Если $g(.)$ — функция активации,
`sigmoid_backward` и `relu_backward` вычисляют $$dZ^{[l]} = dA^{[l]} * g'(Z^{[l]}) \tag{11}$$.

**Упражнение**. Реализуйте обратное распространение ошибки для слоя *LINEAR->ACTIVATION*.

In [37]:
# GRADED FUNCTION: linear_activation_backward

def linear_activation_backward(dA, cache, activation):
    """
    Implement the backward propagation for the LINEAR->ACTIVATION layer.
    
    Arguments:
    dA -- post-activation gradient for current layer l 
    cache -- tuple of values (linear_cache, activation_cache) we store for computing backward propagation efficiently
    activation -- the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"
    
    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b
    """
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ = relu_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
        ### END CODE HERE ###
        
    elif activation == "sigmoid":
        ### START CODE HERE ### (≈ 2 lines of code)
        dZ = sigmoid_backward(dA,activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
        ### END CODE HERE ###
    
    return dA_prev, dW, db

In [38]:
dAL, linear_activation_cache = linear_activation_backward_test_case()

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "sigmoid")
print ("sigmoid:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db) + "\n")

dA_prev, dW, db = linear_activation_backward(dAL, linear_activation_cache, activation = "relu")
print ("relu:")
print ("dA_prev = "+ str(dA_prev))
print ("dW = " + str(dW))
print ("db = " + str(db))

sigmoid:
dA_prev = [[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]]
dW = [[ 0.10266786  0.09778551 -0.01968084]]
db = [[-0.05729622]]

relu:
dA_prev = [[ 0.44090989 -0.        ]
 [ 0.37883606 -0.        ]
 [-0.2298228   0.        ]]
dW = [[ 0.44513824  0.37371418 -0.10478989]]
db = [[-0.20837892]]


**Ожидаемый вывод для sigmoid:**

<table style="width:100%">
  <tr>
    <td > dA_prev </td> 
           <td >[[ 0.11017994  0.01105339]
 [ 0.09466817  0.00949723]
 [-0.05743092 -0.00576154]] </td> 
  </tr> 
    <tr>
    <td > dW </td> 
           <td > [[ 0.10266786  0.09778551 -0.01968084]] </td> 
  </tr> 
    <tr>
    <td > db </td> 
           <td > [[-0.05729622]] </td> 
  </tr> 
</table>



**Ожидаемый вывод для relu:**

<table style="width:100%">
  <tr>
    <td > dA_prev </td> 
           <td > [[ 0.44090989  0.        ]
 [ 0.37883606  0.        ]
 [-0.2298228   0.        ]] </td> 
  </tr> 
    <tr>
    <td > dW </td> 
           <td > [[ 0.44513824  0.37371418 -0.10478989]] </td> 
  </tr> 
    <tr>
    <td > db </td> 
           <td > [[-0.20837892]] </td> 
  </tr> 
</table>



### 6.3 - L-модель назад

Теперь вы реализуете обратную функцию для всей сети. Напомним, что когда вы реализовали функцию L_model_forward, на каждой итерации вы сохраняли кеш, содержащий (X,W,b и z). В модуле обратного распространения вы будете использовать эти переменные для вычисления градиентов. Поэтому в функции `L_model_backward` вы будете перебирать все скрытые слои в обратном порядке, начиная со слоя $L$. На каждом этапе вы будете использовать кэшированные значения слоя $l$ для обратного распространения данных по слою $l$. На рисунке 5 ниже показан обратный проход.

<img src="images/mn_backward.png" style="width:450px;height:300px;">
<caption><center><bold>Рис. 5</bold>: Обратный проход </center></caption>

**Инициализация обратного распространения ошибки**:  
Чтобы выполнить обратное распространение, мы знаем, что выход сети будут следующим:
$A^{[L]} = \sigma(Z^{[L]})$. Таким образом, вашему коду необходимо вычислить `dAL` $= \frac{\partial \mathcal{L}}{\partial A^{[L]}}$.
Для этого используйте формулу:
``` питон
dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL)) # производная стоимости по AL
```

Затем вы можете использовать этот градиент после активации «dAL», чтобы продолжить движение назад. Как видно на рисунке 5, теперь вы можете ввести dAL в реализованную вами обратную функцию LINEAR->SIGMOID (которая будет использовать кэшированные значения, сохраненные функцией L_model_forward). После этого вам придется использовать цикл for для перебора всех остальных слоев с помощью обратной функции LINEAR->RELU. Вы должны хранить каждый dA, dW и db в словаре градиентов. Для этого используйте эту формулу:

$$grads["dW" + str(l)] = dW^{[l]}\tag{15} $$

Например, для $l=3$ это сохранит $dW^{[l]}$ в `grads["dW3"]`.

**Упражнение**: Реализуйте обратное распространение ошибки для модели *[LINEAR->RELU] $\times$ (L-1) -> LINEAR -> SIGMOID*.

In [57]:
# GRADED FUNCTION: L_model_backward

def L_model_backward(AL, Y, caches):
    """
    Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 
    """
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    ### START CODE HERE ### (1 line of code)
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    ### END CODE HERE ###
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    ### START CODE HERE ### (approx. 2 lines)
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, activation="sigmoid")
    ### END CODE HERE ###
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        ### START CODE HERE ### (approx. 5 lines)
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(L-1)], current_cache, activation = "relu")
        grads["dA" + str(l + 1)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp
        ### END CODE HERE ###

    return grads

In [58]:
AL, Y_assess, caches = L_model_backward_test_case()
grads = L_model_backward(AL, Y_assess, caches)
print_grads(grads)

dW1 = [[0.41010002 0.07807203 0.13798444 0.10502167]
 [0.         0.         0.         0.        ]
 [0.05283652 0.01005865 0.01777766 0.0135308 ]]
db1 = [[-0.22007063]
 [ 0.        ]
 [-0.02835349]]
dA1 = [[ 0.          0.52257901]
 [ 0.         -0.3269206 ]
 [ 0.         -0.32070404]
 [ 0.         -0.74079187]]


**Ожидаемый вывод**

<table style="width:60%">
  <tr>
    <td > dW1 </td> 
           <td > [[ 0.41010002  0.07807203  0.13798444  0.10502167]
 [ 0.          0.          0.          0.        ]
 [ 0.05283652  0.01005865  0.01777766  0.0135308 ]] </td> 
  </tr> 
    <tr>
    <td > db1 </td> 
           <td > [[-0.22007063]
 [ 0.        ]
 [-0.02835349]] </td> 
  </tr> 
  <tr>
  <td > dA1 </td> 
           <td > [[ 0.12913162 -0.44014127]
 [-0.14175655  0.48317296]
 [ 0.01663708 -0.05670698]] </td> 
  </tr> 
</table>



### 6.4 - Обновление параметров (обучение)

В этом разделе вы обновите параметры модели, используя градиентный спуск:

$$ W^{[l]} = W^{[l]} - \alpha \text{ } dW^{[l]} \tag{16}$$
$$ b^{[l]} = b^{[l]} - \alpha \text{ } db^{[l]} \tag{17}$$

где $\alpha$ — скорость обучения. После вычисления обновленных параметров сохраните их в словаре параметров.

**Упражнение**. Реализуйте `update_parameters()`, чтобы обновить параметры с помощью градиентного спуска.

**Инструкции**:  
Обновите параметры, используя градиентный спуск для каждых $W^{[l]}$ и $b^{[l]}$ для $l = 1, 2, ..., L$.

In [59]:
# GRADED FUNCTION: update_parameters

def update_parameters(parameters, grads, learning_rate):
    """
    Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...
    """
    
    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    ### START CODE HERE ### (≈ 3 lines of code)
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW"+ str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db"+ str(l+1)]
    ### END CODE HERE ###
    return parameters

In [60]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads, 0.1)

print ("W1 = "+ str(parameters["W1"]))
print ("b1 = "+ str(parameters["b1"]))
print ("W2 = "+ str(parameters["W2"]))
print ("b2 = "+ str(parameters["b2"]))

W1 = [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]]
b1 = [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]]
W2 = [[-0.55569196  0.0354055   1.32964895]]
b2 = [[-0.84610769]]


**Ожидаемый вывод**:

<table style="width:100%"> 
    <tr>
    <td > W1 </td> 
           <td > [[-0.59562069 -0.09991781 -2.14584584  1.82662008]
 [-1.76569676 -0.80627147  0.51115557 -1.18258802]
 [-1.0535704  -0.86128581  0.68284052  2.20374577]] </td> 
  </tr> 
    <tr>
    <td > b1 </td> 
           <td > [[-0.04659241]
 [-1.28888275]
 [ 0.53405496]] </td> 
  </tr> 
  <tr>
    <td > W2 </td> 
           <td > [[-0.55569196  0.0354055   1.32964895]]</td> 
  </tr> 
    <tr>
    <td > b2 </td> 
           <td > [[-0.84610769]] </td> 
  </tr> 
</table>


## 7 - Заключение

Поздравляем с реализацией всех функций, необходимых для построения глубокой нейронной сети!

Мы знаем, что это было долгое задание, но в дальнейшем оно будет только лучше. Следующая часть задания проще.

В следующем задании вы соедините все это вместе, чтобы построить две модели:
- Двухслойная нейронная сеть
- Нейронная сеть из L слоёв.

Фактически вы будете использовать эти модели для классификации изображений кошек и не кошек!